# Modélisation 

### Import des modules 

In [237]:
import numpy as np

# Selection
from sklearn.model_selection import (
    cross_validate,
)

# Preprocess
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Modèles
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import sys
import importlib

sys.path.append("..")
import config
importlib.reload(config)  # Ensure we get the latest TARGET value
from config import TARGET

In [238]:
import pandas as pd
building_consumption = pd.read_csv("../assets/building_consumption_cleaned.csv")
building_consumption.shape

(1539, 26)

### Comparaison de différents modèles supervisés

A réaliser :
* Pour chaque algorithme que vous allez tester, vous devez :
    * Réaliser au préalable une séparation en jeu d'apprentissage et jeu de test via une validation croisée.
    * Si les features quantitatives que vous souhaitez utiliser ont des ordres de grandeur très différents les uns des autres, et que vous utilisez un algorithme de regression qui est sensible à cette différence, alors il faut réaliser un scaling (normalisation) de la donnée au préalable.
    * Entrainer le modèle sur le jeu de Train
    * Prédire la cible sur la donnée de test (nous appelons cette étape, l'inférence).
    * Calculer les métriques de performance R2, MAE et RMSE sur le jeu de train et de test.
    * Interpréter les résultats pour juger de la fiabilité de l'algorithme.
* Vous pouvez choisir par exemple de tester un modèle linéaire, un modèle à base d'arbres et un modèle de type SVM
* Déterminer le modèle le plus performant parmi ceux testés.

### Modèle linéaire - LinearRegression

In [239]:
# Select features (excluding the target column)
features_df = building_consumption.drop(columns=[TARGET])

numerical_features = features_df.select_dtypes(include="number").columns.tolist()
categorical_features = features_df.select_dtypes(exclude="number").columns.tolist()

print("Numerical features:", len(numerical_features))
print("Categorical features:", len(categorical_features))
print("Total features:", len(numerical_features) + len(categorical_features))
print("Total columns (excluding target):", len(features_df.columns))
print("Match:", len(features_df.columns) == len(numerical_features) + len(categorical_features))

print("\nNumerical features:", numerical_features)
print("\nCategorical features:", categorical_features)


Numerical features: 25
Categorical features: 0
Total features: 25
Total columns (excluding target): 25
Match: True

Numerical features: ['NumberofBuildings', 'PropertyGFAParking', 'PropertyGFABuilding(s)', 'ENERGYSTARScore', 'NumberOfUseTypes', 'SteamUse', 'NaturalGas', 'Electricity', 'PrimaryPropertyType_0', 'PrimaryPropertyType_1', 'PrimaryPropertyType_2', 'PrimaryPropertyType_3', 'PrimaryPropertyType_4', 'BuildingType_Nonresidential COS', 'BuildingType_Nonresidential WA', 'BuildingType_SPS-District K-12', 'NumberofFloors_quintile_Q2', 'NumberofFloors_quintile_Q3', 'NumberofFloors_quintile_Q4', 'NumberofFloors_quintile_Q5', 'EraBuilt_1951-1970', 'EraBuilt_1971-1990', 'EraBuilt_1991-2010', 'EraBuilt_Post-2010', 'EraBuilt_≤1930']

Categorical features: []


**Note importante:** Pour une comparaison équitable, tous les modèles utilisent la même stratégie de validation croisée avec `KFold(n_splits=5, shuffle=True, random_state=42)`. Cela garantit que chaque modèle est évalué sur exactement les mêmes divisions train/test.

### Configuration commune pour l'évaluation des modèles

In [240]:
# Configuration commune pour tous les modèles
from sklearn.model_selection import KFold

# Stratégie de validation croisée commune pour tous les modèles
cv_strategy = KFold(n_splits=5, shuffle=True, random_state=42)

# Métriques d'évaluation communes
scoring = ["r2", "neg_mean_squared_error", "neg_mean_absolute_error"]

# Données communes
X = building_consumption.drop(columns=[TARGET])
y = building_consumption[TARGET]

def evaluate_model(pipeline, model_name):
    """
    Évalue un modèle avec la stratégie de validation croisée commune
    
    Args:
        pipeline: Le pipeline sklearn à évaluer
        model_name: Nom du modèle pour l'affichage
        
    Returns:
        dict: Résultats de la validation croisée
    """
    cv_results = cross_validate(
        pipeline,
        X,
        y,
        cv=cv_strategy,
        scoring=scoring,
        return_train_score=True,
    )
    
    print(f"=== {model_name} Results ===")
    print(f"{'Metric':<12} {'Train Mean':<12} {'Train Std':<12} {'Test Mean':<12} {'Test Std':<12}")
    print("-" * 65)
    
    # Affichage des résultats moyens avec format tabulaire
    for metric in scoring:
        train_scores = cv_results[f"train_{metric}"]
        test_scores = cv_results[f"test_{metric}"]
        # Inverser les scores négatifs pour MAE et RMSE
        if "neg" in metric:
            train_scores = -train_scores
            test_scores = -test_scores
            metric_name = metric.replace("neg_", "").replace("_", " ").upper()
        else:
            metric_name = metric.upper()
        
        print(f"{metric_name:<12} {train_scores.mean():<12.3f} {train_scores.std():<12.3f} {test_scores.mean():<12.3f} {test_scores.std():<12.3f}")
    
    # Calculate RMSE from MSE
    rmse_train = np.sqrt(-cv_results["train_neg_mean_squared_error"])
    rmse_test = np.sqrt(-cv_results["test_neg_mean_squared_error"])
    print(f"{'RMSE':<12} {rmse_train.mean():<12.3f} {rmse_train.std():<12.3f} {rmse_test.mean():<12.3f} {rmse_test.std():<12.3f}")
    print()  # Ligne vide pour séparation
    
    return cv_results

In [241]:
# Since all features are numerical (categorical ones were already encoded) 
# and missing values have been cleaned in the data preparation notebook,
# we only need to scale the numerical features
if len(categorical_features) > 0:
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), numerical_features),
            ("cat", "passthrough", categorical_features),
        ]
    )
else:
    # All features are numerical and clean, just apply scaling
    preprocessor = StandardScaler()

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

# Évaluation avec la fonction commune
cv_results = evaluate_model(pipeline, "LinearRegression")

=== LinearRegression Results ===
Metric       Train Mean   Train Std    Test Mean    Test Std    
-----------------------------------------------------------------
R2           0.451        0.025        0.363        0.105       
MEAN SQUARED ERROR 178631.446   12086.509    195869.913   52794.949   
MEAN ABSOLUTE ERROR 176.095      6.083        181.801      14.133      
RMSE         422.407      14.276       438.370      60.842      



### Modèle SVR (Support Vector Regressor)

In [242]:
# SVR is sensitive to feature scaling, so we need StandardScaler
pipeline_svr = Pipeline(steps=[
    ('preprocessor', StandardScaler()),
    ('model', SVR(kernel='rbf', C=1.0, gamma='scale'))
])

# Évaluation avec la fonction commune
cv_results_svr = evaluate_model(pipeline_svr, "SVR")

=== SVR Results ===
Metric       Train Mean   Train Std    Test Mean    Test Std    
-----------------------------------------------------------------
R2           -0.022       0.002        -0.024       0.009       
MEAN SQUARED ERROR 333567.842   33884.583    333744.802   136324.466  
MEAN ABSOLUTE ERROR 134.183      2.412        134.902      10.082      
RMSE         576.790      29.681       565.753      116.911     



### Modèle RandomForestRegressor

In [243]:
# Random Forest doesn't require feature scaling, but let's use it for consistency
pipeline_rf = Pipeline(steps=[
    # ('preprocessor', StandardScaler()),
    ('model', RandomForestRegressor(n_estimators=500, random_state=42, max_depth=20, min_samples_split=2))
])

# Évaluation avec la fonction commune
cv_results_rf = evaluate_model(pipeline_rf, "Random Forest")

=== Random Forest Results ===
Metric       Train Mean   Train Std    Test Mean    Test Std    
-----------------------------------------------------------------
R2           0.930        0.008        0.551        0.128       
MEAN SQUARED ERROR 23046.386    3788.055     135034.233   40369.397   
MEAN ABSOLUTE ERROR 42.413       1.055        109.535      5.722       
RMSE         151.310      12.320       363.511      53.797      



### Modèle GradientBoostingRegressor

In [244]:
# GradientBoostingRegressor
pipeline_gb = Pipeline(steps=[
	# ('preprocessor', StandardScaler()),
	('model', GradientBoostingRegressor(n_estimators=300, random_state=42, max_depth=3, learning_rate=0.05, subsample=1.0))
])

# Évaluation avec la fonction commune
cv_results_gb = evaluate_model(pipeline_gb, "Gradient Boosting")

=== Gradient Boosting Results ===
Metric       Train Mean   Train Std    Test Mean    Test Std    
-----------------------------------------------------------------
R2           0.966        0.006        0.657        0.206       
MEAN SQUARED ERROR 11044.831    1151.760     105598.934   63015.261   
MEAN ABSOLUTE ERROR 60.788       2.372        106.973      7.832       
RMSE         104.951      5.481        309.677      98.485      



### Comparaison des 4 modèles

In [245]:
# Create comparison table
import pandas as pd

# Extract results from cross-validation
models_comparison = pd.DataFrame({
    'Model': ['LinearRegression', 'SVR', 'RandomForest', 'GradientBoosting'],
    'R2_Train': [
        cv_results["train_r2"].mean(),
        cv_results_svr["train_r2"].mean(),
        cv_results_rf["train_r2"].mean(),
        cv_results_gb["train_r2"].mean()
    ],
    'R2_Test': [
        cv_results["test_r2"].mean(),
        cv_results_svr["test_r2"].mean(),
        cv_results_rf["test_r2"].mean(),
        cv_results_gb["test_r2"].mean()
    ],
    'MAE_Train': [
        (-cv_results["train_neg_mean_absolute_error"]).mean(),
        (-cv_results_svr["train_neg_mean_absolute_error"]).mean(),
        (-cv_results_rf["train_neg_mean_absolute_error"]).mean(),
        (-cv_results_gb["train_neg_mean_absolute_error"]).mean()
    ],
    'MAE_Test': [
        (-cv_results["test_neg_mean_absolute_error"]).mean(),
        (-cv_results_svr["test_neg_mean_absolute_error"]).mean(),
        (-cv_results_rf["test_neg_mean_absolute_error"]).mean(),
        (-cv_results_gb["test_neg_mean_absolute_error"]).mean()
    ],
    'RMSE_Train': [
        np.sqrt(-cv_results["train_neg_mean_squared_error"]).mean(),
        np.sqrt(-cv_results_svr["train_neg_mean_squared_error"]).mean(),
        np.sqrt(-cv_results_rf["train_neg_mean_squared_error"]).mean(),
        np.sqrt(-cv_results_gb["train_neg_mean_squared_error"]).mean()
    ],
    'RMSE_Test': [
        np.sqrt(-cv_results["test_neg_mean_squared_error"]).mean(),
        np.sqrt(-cv_results_svr["test_neg_mean_squared_error"]).mean(),
        np.sqrt(-cv_results_rf["test_neg_mean_squared_error"]).mean(),
        np.sqrt(-cv_results_gb["test_neg_mean_squared_error"]).mean()
    ]
})

# Format the table for better readability
models_comparison = models_comparison.round(3)
print("=== COMPARAISON DES MODÈLES ===")
print(models_comparison.to_string(index=False))

# Find best model for each metric
print("\n=== MEILLEUR MODÈLE PAR MÉTRIQUE ===")
print(f"Meilleur R² Test: {models_comparison.loc[models_comparison['R2_Test'].idxmax(), 'Model']} (R² = {models_comparison['R2_Test'].max():.3f})")
print(f"Meilleur MAE Test: {models_comparison.loc[models_comparison['MAE_Test'].idxmin(), 'Model']} (MAE = {models_comparison['MAE_Test'].min():.0f})")
print(f"Meilleur RMSE Test: {models_comparison.loc[models_comparison['RMSE_Test'].idxmin(), 'Model']} (RMSE = {models_comparison['RMSE_Test'].min():.0f})")

# Calculate overfitting indicator (difference between train and test R²)
models_comparison['Overfitting'] = models_comparison['R2_Train'] - models_comparison['R2_Test']
print(f"\n=== ANALYSE DU SURAPPRENTISSAGE (Train R² - Test R²) ===")
for i, row in models_comparison.iterrows():
    print(f"{row['Model']}: {row['Overfitting']:.3f}")
    
print(f"\nModèle le moins sujet au surapprentissage: {models_comparison.loc[models_comparison['Overfitting'].idxmin(), 'Model']}")

=== COMPARAISON DES MODÈLES ===
           Model  R2_Train  R2_Test  MAE_Train  MAE_Test  RMSE_Train  RMSE_Test
LinearRegression     0.451    0.363    176.095   181.801     422.407    438.370
             SVR    -0.022   -0.024    134.183   134.902     576.790    565.753
    RandomForest     0.930    0.551     42.413   109.535     151.310    363.511
GradientBoosting     0.966    0.657     60.788   106.973     104.951    309.677

=== MEILLEUR MODÈLE PAR MÉTRIQUE ===
Meilleur R² Test: GradientBoosting (R² = 0.657)
Meilleur MAE Test: GradientBoosting (MAE = 107)
Meilleur RMSE Test: GradientBoosting (RMSE = 310)

=== ANALYSE DU SURAPPRENTISSAGE (Train R² - Test R²) ===
LinearRegression: 0.088
SVR: 0.002
RandomForest: 0.379
GradientBoosting: 0.309

Modèle le moins sujet au surapprentissage: SVR
